# **Segmentação de MRI Cardíacas**



# Dependências

Bibliotecas

In [ ]:
!pip install tensorflow==2.5.0

!pip install scikit-learn==0.24.2
!pip install segmentation-models==1.0.1

!pip install nibabel==3.2.1
!pip install MedPy==0.4.0

!pip install matplotlib

!pip install python-decouple==3.4

!pip install pydot

Acesso à arquivos

In [ ]:
from decouple import Csv, config

In [ ]:
COLAB_MAIN_PATH = '/content/drive, My Drive/Colab Notebooks/data'

In [ ]:
MAIN_PATH = config('MAIN_PATH', default=COLAB_MAIN_PATH, cast=Csv())
MAIN_PATH

In [ ]:
if config('IN_GOOGLE_COLAB', default=True, cast=bool):
    import os, sys
    from google.colab import drive

    drive.mount(MAIN_PATH[0], force_remount=True)
    os.chdir('/'.join(MAIN_PATH))
    sys.path.append('/'.join(MAIN_PATH))

Visualização da GPU disponibilizada

In [ ]:
!nvidia-smi

Visualizando a quantidade de GPUs detectadas

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(f'GPUs available: {len(gpus)}')

Configuração de uso gradual e necessário da memória da GPU utilizada

In [ ]:
try:
    # Currently, memory growth needs to be the same across GPUs
    for device in gpus:
        tf.config.experimental.set_memory_growth(device, True)

    logical_devices = tf.config.list_logical_devices(device_type)
    print(f'{len(gpus)} physical vs. {len(logical_devices)} logical')
    
except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
del gpus
del logical_devices

# Constantes

In [ ]:
from os.path import join as join_path

In [ ]:
IMG_SZ = 100

In [ ]:
RANDOM_SEED = 42

Imagens

In [ ]:
# represent voxels located in the:
# 0: background
# 1: RV cavity
# 2: myocardium
# 3: LV cavity 
CLASSES_CNT = 4

In [ ]:
INPUT_CHANNELS = 1

In [ ]:
PATCH_STEP = 30

In [ ]:
IMG_SUFIX = '_data'

In [ ]:
MASK_SUFIX = '_mask'

In [ ]:
CLASS_WEIGHT_FILENAME = 'classes_weight.npy'
CLASS_WEIGHT_FILENAME = join_path(*MAIN_PATH, CLASS_WEIGHT_FILENAME)

In [ ]:
RAW_DATA_PRE_PROCESS_SZ = 3

Data-subgroup

In [ ]:
VALIDATION_PROPORTION = .25

In [ ]:
XVALIDATION_PROPORTION = .25

Datasets

In [ ]:
DT_SHUFFLE_SZ = 50

In [ ]:
BASE_DATA_DIR = 'ACDC_2017_dados'

RAW_TRAINING_FILENAME = 'training.zip'
RAW_TRAINING_DIR = 'raw_train'
PATCHES_DIR = f'image_patches_{IMG_SZ}'
TRAINING_DIR = 'train'
X_VAL_DIR = 'validation'
TEST_DIR = 'test'

RAW_TRAINING_DIR = join_path(BASE_DATA_DIR, RAW_TRAINING_DIR)
TRAINING_DIR = join_path(BASE_DATA_DIR, PATCHES_DIR, TRAINING_DIR)
X_VAL_DIR = join_path(BASE_DATA_DIR, PATCHES_DIR, X_VAL_DIR)
TEST_DIR = join_path(BASE_DATA_DIR, PATCHES_DIR, TEST_DIR)

RAW_TRAINING_FILENAME = join_path(*MAIN_PATH, RAW_TRAINING_FILENAME)
RAW_TRAINING_DIR = join_path(*MAIN_PATH, RAW_TRAINING_DIR)
TRAINING_DIR = join_path(*MAIN_PATH, TRAINING_DIR)
X_VAL_DIR = join_path(*MAIN_PATH, X_VAL_DIR)
TEST_DIR = join_path(*MAIN_PATH, TEST_DIR)

In [ ]:
IMG_GLOB_SEARCH = ''.join(['*', IMG_SUFIX, '.npy'])

In [ ]:
MASK_GLOB_SEARCH = ''.join(['*', MASK_SUFIX, '.npy'])

Modelos

In [ ]:
# MODEL_FILENAME = 'ulas_model'
# MODEL_FILENAME = 'ulas_model_v2'
# MODEL_FILENAME = 'ulas_model_dice_loss'
MODEL_FILENAME = 'ulas_model_dice_loss'

In [ ]:
MODEL_DIR = join_path(*MAIN_PATH, MODEL_FILENAME)

Arquitetura Ulas

In [ ]:
POOL_SIZE = 5

Métricas e Funções de perda

In [ ]:
SMOOTH = 1e-3

In [ ]:
FOCAL_LOSS_GAMA = 2

Treinamento

In [ ]:
INIT_LEARNING_RATE = 1e-3

In [ ]:
MIN_LEARNING_RATE = 1e-6

In [ ]:
BATCH_SIZE = 4

In [ ]:
INITAL_EPOCH = 0

In [ ]:
MAX_EPOCH = 10

In [ ]:
MAIN_METRIC = 'dice_metric'

In [ ]:
if MODEL_FILENAME == 'unet_plusplus_model':
    EVAL_METRIC = f're_lu_{MAIN_METRIC}'
else:
    EVAL_METRIC = MAIN_METRIC

In [ ]:
METRIC_THRESHOLD = .5

Data augmentation

In [ ]:
AUG_ROTATION = 90

In [ ]:
AUG_W_HOR_FLIP = True

In [ ]:
AUG_W_VER_FLIP = True

In [ ]:
# AUG_WIDTH_RNG = .4

In [ ]:
# AUG_HEIGHT_RNG = .4

In [ ]:
# AUG_ZOOM_FACTOR = 10
# AUG_ZOOM = (10, 16) # x10

In [ ]:
AUG_FILL_MODE = 'constant'

Milestones do treinamento

In [ ]:
EARLY_STOP_PATIENCE = 5

In [ ]:
LR_REDUCER_PATIENCE = 3

In [ ]:
STEPS_FILENAME = join_path(
    MODEL_DIR,
    f'{MODEL_FILENAME}.h5'
)

In [ ]:
CSV_LOG_FILENAME = join_path(MODEL_DIR, f'{MODEL_FILENAME}_log.csv')

Resultados

In [ ]:
PLOT_FILENAME = f'{MODEL_FILENAME}_structure.png'
PLOT_FILENAME = join_path(MODEL_DIR, PLOT_FILENAME)

In [ ]:
PLOT_WIDTH = 720

# Manipulação das imagens

Imports desta seção

In [ ]:
import os

import nibabel as nib
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.image import extract_patches

Imagens originais, .nii.gz

In [ ]:
"""
author: Clément Zotti (clement.zotti@usherbrooke.ca)
date: April 2017

DESCRIPTION :
The script provide helpers functions to handle nifti image format:
    - load_nii()
    - save_nii()

to generate metrics for two images:
    - metrics()

And it is callable from the command line (see below).
Each function provided in this script has comments to understand
how they works.

HOW-TO:

This script was tested for python 3.4.

First, you need to install the required packages with
    pip install -r requirements.txt

After the installation, you have two ways of running this script:
    1) python metrics.py ground_truth/patient001_ED.nii.gz prediction/patient001_ED.nii.gz
    2) python metrics.py ground_truth/ prediction/

The first option will print in the console the dice and volume of each class for the given image.
The second option wiil ouput a csv file where each images will have the dice and volume of each class.


Based on: http://acdc.creatis.insa-lyon.fr
"""
#
# Utils function to load and save nifti files with the nibabel package
#
def load_nii(img_path):
    """
    Function to load a 'nii' or 'nii.gz' file, The function returns
    everyting needed to save another 'nii' or 'nii.gz'
    in the same dimensional space, i.e. the affine matrix and the header

    Parameters
    ----------

    img_path: string
    String with the path of the 'nii' or 'nii.gz' image file name.

    Returns
    -------
    Three element, the first is a numpy array of the image values,
    the second is the affine transformation of the image, and the
    last one is the header of the image.
    """
    nimg = nib.load(img_path)
    return nimg.get_fdata().astype('float32'), nimg.affine, nimg.header

def save_nii(img_path, data, affine, header):
    """
    Function to save a 'nii' or 'nii.gz' file.

    Parameters
    ----------

    img_path: string
    Path to save the image should be ending with '.nii' or '.nii.gz'.

    data: np.array
    Numpy array of the image data.

    affine: list of list or np.array
    The affine transformation to save with the image.

    header: nib.Nifti1Header
    The header that define everything about the data
    (pleasecheck nibabel documentation).
    """
    nimg = nib.Nifti1Image(data, affine=affine, header=header)
    nimg.to_filename(img_path)

Localização dos arquivos

In [ ]:
def file_paths(base_dir):
    c_paths = {
        'config': [],
        'ground_truth': [],
        'images': [],
        '4d': []
    }

    for c_dir, next_dirs, file_names in os.walk(base_dir):
        # skip root
        if next_dirs:
            continue

        c_patient = os.path.basename(c_dir)

        for f_n in file_names:
            f_n = os.path.join(c_dir, f_n)

            if f_n.endswith('.cfg'): file_type = 'config'
            elif '_gt' in f_n: file_type = 'ground_truth'
            elif '_4d' in f_n: file_type = '4d'
            else: file_type = 'images'

            c_paths[file_type].append(f_n)

    for file_type in c_paths.keys():
        c_paths[file_type].sort()

    return c_paths

Conversões diretas salvas

In [ ]:
def load_data(file_paths, is_training=True, initial=0, look_up_sz=0):
    images_filename = os.path.join(*MAIN_PATH, f'{"training" if is_training else "test"}_data.npy')
    gt_filename = os.path.join(*MAIN_PATH, f'{"training" if is_training else "test"}_gt_data.npy')

    try:
        imgs = np.load(images_filename, allow_pickle=True)
        imgs_gt = np.load(gt_filename, allow_pickle=True)

        if np.any(imgs) and np.any(imgs_gt):
            print('carregado dos arquivos')

            return tf.convert_to_tensor(imgs), tf.convert_to_tensor(imgs_gt, dtype=tf.float32)
    except:
        print('lendo imagens arquivos')
        imgs, imgs_gt =  _load_data(file_paths, initial, look_up_sz)

        # np.save(images_filename, imgs)
        # np.save(gt_filename, imgs_gt)

        return imgs, imgs_gt

def load_classes_weight(labels=None):
    return None
    try:
        weights = np.load(CLASS_WEIGHT_FILENAME, allow_pickle=True)

        if np.any(weights):
            print('carregado do arquivo')

            return weights
    except:
        print('calculando pesos')
        if not np.any(labels):
            return None

        weights = compute_class_weight(
            'balanced',
            np.arange(CLASSES_CNT),
            tf.reshape(tf.cast(labels, tf.int8), [-1]).numpy()
        )

        np.save(CLASS_WEIGHT_FILENAME, weights)

        return weights

Pré-processamento das imagens originais

In [ ]:
def _load_data(file_paths, initial=0, look_up_sz=0):
    images, ground_truth = tf.convert_to_tensor([]), tf.convert_to_tensor([])
    cnt = initial
    
    final_idx = min(initial + look_up_sz, len(file_paths['images']))

    raw_images = file_paths['images'][initial: final_idx]
    raw_ground_truth = file_paths['ground_truth'][initial: final_idx]

    for c_img, c_ground_truth in zip(raw_images, raw_ground_truth):
        print(f'processing {cnt}...')

        # we load 3D training image
        training_image, _, _ = load_nii(c_img)
        # we load 3D training mask (shape=(512,512,129))
        train_ground_truth, _, _ = load_nii(c_ground_truth)

        for k in range(min(train_ground_truth.shape[-1], training_image.shape[-1])):
            #axial cuts are made along the z axis with undersampling
            gt_2d = np.array(train_ground_truth[::, ::, k])

            # invalid ground truth
            if len(np.unique(gt_2d)) == 1:
                continue

            image_patches = _extract_patches(
                _pre_process_img(np.array(training_image[::, ::, k]))
            )
            gt_patches = _extract_patches(gt_2d)

            if (tf.size(images) == 0).numpy():
                images = image_patches
            else:
                images = tf.concat((images, image_patches), axis=0)

            if (tf.size(ground_truth) == 0).numpy():
                ground_truth = gt_patches
            else:
                ground_truth = tf.concat((ground_truth, gt_patches), axis=0)

        cnt += 1

    ground_truth = tf.cast(ground_truth, tf.int8)

    return images, ground_truth

def _extract_patches(image_2d):
    image_2d = tf.expand_dims(tf.expand_dims(image_2d, axis=-1), axis=0)

    if image_2d.shape[1] < IMG_SZ or image_2d.shape[2] < IMG_SZ:
        return tf.image.resize(
            image_2d, (IMG_SZ, IMG_SZ), method='nearest'
        )

    image_patches = extract_patches(
        image_2d,
        sizes=[1, IMG_SZ, IMG_SZ, 1],
        strides=[1, PATCH_STEP, PATCH_STEP, 1],
        rates=[1, 1, 1, 1],
        padding='VALID'
    )

    return tf.reshape(
        image_patches,
        [image_patches.shape[1] * image_patches.shape[2], IMG_SZ, IMG_SZ, 1]
    )

# Based on: https://github.com/MinaJf/FU-net/blob/HEAD/image_loder.py
def _pre_process_img(img):
    img = tf.cast(img, tf.float32)

    # standardization (zero mean)
    img -= tf.math.reduce_mean(img)
    img /= tf.math.reduce_std(img)

    # normalize between [0, 1]
    img -= tf.math.reduce_min(img)
    img /= tf.math.reduce_max(img)

    return img

In [ ]:
def save_patches(patches, base_dir, is_label=False, initial=1):
    for ind, img in enumerate(patches, start=initial):
        img_name = f'{ind}{MASK_SUFIX if is_label else IMG_SUFIX}'
        img_path = os.path.join(base_dir, img_name)

        np.save(img_path, img)

        print(f'saved <{patches.dtype}> {img_path}')

In [ ]:
def create_data():
    training_paths = file_paths(RAW_TRAINING_DIR)

    train_patch_cnt, xval_patch_cnt, test_patch_cnt = 1, 1, 1

    for ind in range(0, len(training_paths['images']), RAW_DATA_PRE_PROCESS_SZ):
        raw_train_data, raw_train_labels = load_data(training_paths, initial=ind, look_up_sz=RAW_DATA_PRE_PROCESS_SZ)

        print(f'raw_train_data: {raw_train_data.shape}\n\t{raw_train_data.dtype}')
        print(f'raw_train_labels: {raw_train_labels.shape}\n\t{raw_train_labels.dtype}')

        train_data, test_data, train_labels, test_labels = train_test_split(
            raw_train_data.numpy(), raw_train_labels.numpy(),
            test_size=VALIDATION_PROPORTION,
            random_state=RANDOM_SEED
        )

        train_data, xval_data, train_labels, xval_labels = train_test_split(
            train_data, train_labels,
            test_size=XVALIDATION_PROPORTION,
            random_state=RANDOM_SEED
        )

        print(f'train_data: {train_data.shape} - {train_data.dtype}')
        print(f'train_labels: {train_labels.shape} - {train_labels.dtype}')
        
        print(f'xval_data: {xval_data.shape} - {xval_data.dtype}')
        print(f'xval_labels: {xval_labels.shape} - {xval_labels.dtype}')

        print(f'test_data: {test_data.shape} - {test_data.dtype}')
        print(f'test_labels: {test_labels.shape} - {test_labels.dtype}')

        save_patches(train_labels, TRAINING_DIR, is_label=True, initial=train_patch_cnt)
        save_patches(train_data, TRAINING_DIR, initial=train_patch_cnt)

        save_patches(xval_labels, X_VAL_DIR, is_label=True, initial=xval_patch_cnt)
        save_patches(xval_data, X_VAL_DIR, initial=xval_patch_cnt)

        save_patches(test_labels, TEST_DIR, is_label=True, initial=test_patch_cnt)
        save_patches(test_data, TEST_DIR, initial=test_patch_cnt)

        train_patch_cnt += len(train_data)
        xval_patch_cnt += len(xval_data)
        test_patch_cnt += len(test_data)

        del train_data
        del train_labels

        del xval_data
        del xval_labels
        
        del test_data
        del test_labels
        
    del training_paths


# Manipulação dos dados

In [ ]:
from tensorflow.data import Dataset
from numpy import load
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import skimage.transform as img_transform
from gc import collect as gc_collector
import numpy as np
from tensorflow.keras import backend as K

In [ ]:
# def keras_model_memory_usage_in_bytes(model, *, batch_size: int):
#     """
#     Return the estimated memory usage of a given Keras model in bytes.
#     This includes the model weights and layers, but excludes the dataset.

#     The model shapes are multipled by the batch size, but the weights are not.

#     Args:
#         model: A Keras model.
#         batch_size: The batch size you intend to run the model with. If you
#             have already specified the batch size in the model itself, then
#             pass `1` as the argument here.
#     Returns:
#         An estimate of the Keras model's memory usage in bytes.

#     """
#     default_dtype = tf.keras.backend.floatx()
#     shapes_mem_count = 0
#     internal_model_mem_count = 0
#     for layer in model.layers:
#         if isinstance(layer, tf.keras.Model):
#             internal_model_mem_count += keras_model_memory_usage_in_bytes(
#                 layer, batch_size=batch_size
#             )
#         single_layer_mem = tf.as_dtype(layer.dtype or default_dtype).size
#         out_shape = layer.output_shape
#         if isinstance(out_shape, list):
#             out_shape = out_shape[0]
#         for s in out_shape:
#             if s is None:
#                 continue
#             single_layer_mem *= s
#         shapes_mem_count += single_layer_mem

#     trainable_count = sum(
#         [tf.keras.backend.count_params(p) for p in model.trainable_weights]
#     )
#     non_trainable_count = sum(
#         [tf.keras.backend.count_params(p) for p in model.non_trainable_weights]
#     )

#     total_memory = (
#         batch_size * shapes_mem_count
#         + internal_model_mem_count
#         + trainable_count
#         + non_trainable_count
#     )
#     return total_memory

In [ ]:
# def get_model_memory_usage(batch_size, model):
#     shapes_mem_count = 0
#     internal_model_mem_count = 0
#     for l in model.layers:
#         layer_type = l.__class__.__name__
#         if layer_type == 'Model':
#             internal_model_mem_count += get_model_memory_usage(batch_size, l)
#         single_layer_mem = 1
#         out_shape = l.output_shape
#         if type(out_shape) is list:
#             out_shape = out_shape[0]
#         for s in out_shape:
#             if s is None:
#                 continue
#             single_layer_mem *= s
#         shapes_mem_count += single_layer_mem

#     trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
#     non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

#     print(K.floatx())
#     number_size = 4.0
#     if K.floatx() == 'float16':
#         number_size = 2.0
#     if K.floatx() == 'float64':
#         number_size = 8.0

#     total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
#     gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
#     return gbytes

In [ ]:
def check_cnt(main_path):
    img_filenames = glob(os.path.join(main_path, IMG_GLOB_SEARCH))
    mask_filenames = glob(os.path.join(main_path, MASK_GLOB_SEARCH))

    img_cnt = len(img_filenames)
    mask_cnt = len(mask_filenames)

    print('img_cnt', img_cnt)
    print('mask_cnt', mask_cnt)

    return (
        img_cnt if img_cnt == mask_cnt else 0,
        tf.convert_to_tensor(img_filenames),
    )

In [ ]:
def _load_images(batch_paths):
    def _load_img(img_path):
        img_path = img_path.decode()

        image = load(img_path, allow_pickle=True)
        mask_path = img_path.replace(IMG_SUFIX, MASK_SUFIX)
        
        mask = load(mask_path, allow_pickle=True)

        mask = mask.astype('int8')
      
        return image, mask

    return _load_img(batch_paths)

In [ ]:
@tf.function
def ensure_shape(img_batch, mask_batch):
    img_batch = tf.convert_to_tensor(img_batch)
    mask_batch = tf.convert_to_tensor(mask_batch)

    # img_batch.set_shape((None, IMG_SZ, IMG_SZ, 1))
    # mask_batch.set_shape((None, IMG_SZ, IMG_SZ, 1))

    img_batch.set_shape((IMG_SZ, IMG_SZ, 1))
    mask_batch.set_shape((IMG_SZ, IMG_SZ, 1))

    return img_batch, mask_batch

In [ ]:
@tf.function
def pre_processing(img_batch, mask_batch):
    mask_batch = tf.cast(mask_batch, 'uint8')
    # if not mask_batch.shape:
    #     mask_batch.set_shape(img_batch.shape)

    mask_batch = tf.one_hot(mask_batch, CLASSES_CNT)
    mask_batch = tf.squeeze(mask_batch)
    # mask_batch = tf.squeeze(mask_batch, axis=3)
    
    mask_batch = tf.cast(mask_batch, 'int8')

    return img_batch, mask_batch

In [ ]:
def _augmentation(combined_batches):
    c_img, c_mask = combined_batches

    # horizontal_flip=True,
    if AUG_W_HOR_FLIP and tf.random.uniform([]) < .5:
        c_img = tf.image.flip_left_right(c_img)
        c_mask = tf.image.flip_left_right(c_mask)

    # vertical_flip=True,
    if AUG_W_VER_FLIP and tf.random.uniform([]) < .5:
        c_img = tf.image.flip_up_down(c_img)
        c_mask = tf.image.flip_up_down(c_mask)

    # rotation_range=90
    if tf.random.uniform([]) < .5:
        angle = tf.random.uniform([], -AUG_ROTATION // 2, AUG_ROTATION // 2, dtype=tf.int32).numpy()

        c_img = img_transform.rotate(np.array(c_img), angle, mode=AUG_FILL_MODE)
        c_mask = img_transform.rotate(np.array(c_mask), angle, mode=AUG_FILL_MODE)

    # zoom_range=AUG_ZOOM
    # width_shift_range=.4
    # height_shift_range=.4
    
    return tf.convert_to_tensor(c_img), tf.convert_to_tensor(c_mask, dtype='int8')

In [ ]:
def data_augmentation(img_batch, mask_batch):
    gc_collector()

    return _augmentation((img_batch, mask_batch))

In [ ]:
@tf.function
def final_cast(img_batch, mask_batch):
    mask_batch = tf.cast(mask_batch, 'float32')

    return img_batch, mask_batch

In [ ]:
def get_dataset(main_path, shuffle_sz=False, should_data_aug=False):
    dataset = Dataset.list_files(os.path.join(main_path, IMG_GLOB_SEARCH), shuffle=True, seed=RANDOM_SEED)

    dataset = dataset.repeat()

    if shuffle_sz:
        dataset = dataset.shuffle(
            buffer_size=shuffle_sz,
            reshuffle_each_iteration=True
        )

    dataset = dataset.map(
        lambda img_path: tf.numpy_function(_load_images, [img_path], (tf.float32, tf.int8)),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    dataset = dataset.map(ensure_shape, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.map(pre_processing, num_parallel_calls=tf.data.AUTOTUNE)

    if should_data_aug:
        dataset = dataset.map(
            # data_augmentation,
            lambda img, mask: tf.numpy_function(data_augmentation, [img, mask], (tf.float32, tf.int8)),
            num_parallel_calls=tf.data.AUTOTUNE
        )

    dataset = dataset.map(final_cast, num_parallel_calls=tf.data.AUTOTUNE)

    # dataset = dataset.cache()
    dataset = dataset.batch(BATCH_SIZE)

    dataset = dataset.prefetch(buffer_size=1)

    return dataset

In [ ]:
def display_sample(display_list, to_save=False):
    """Show side-by-side an input image,
    the ground truth and the prediction.
    """
    plt.figure(dpi=180)
    plt.figure(figsize=(10, 10))

    if len(display_list) == 3:
        gt = display_list[1]
        pred = display_list[2]

        # gt = tf.image.resize(gt, (50, 50), 'nearest')
        # pred = tf.image.resize(pred, (50, 50), 'nearest')
        
        equal = gt * pred
        
        display_list.append(equal)
        display_list.append(gt - equal)
        display_list.append(pred - equal)

    settings = [
        { # 0
            'title': 'Input Image',
            'pre_processing': lambda x: tf.squeeze(x, axis=2),
        },
        { # 1
            'title': 'Ground Truth',
            'pre_processing': lambda x: tf.argmax(x, axis=2),
        },
        { # 2
            'title': 'Prediction',
            'pre_processing': lambda x: tf.argmax(x, axis=2),
        },
        { # 3
            'title': 'Match',
            'pre_processing': lambda x: tf.argmax(x, axis=2),
        },
        { # 4
            'title': 'G.T. - Match',
            'pre_processing': lambda x: tf.argmax(x, axis=2),
        },
        { # 5
            'title': 'Pred. - Match',
            'pre_processing': lambda x: tf.argmax(x, axis=2),
        },
    ]

    for ind, image in enumerate(display_list):
        print()
        print(image.shape)
        plt.subplot(1, len(display_list), ind + 1)
        plt.title(settings[ind]['title'])
        plt.imshow(settings[ind]['pre_processing'](image), cmap='hot')
        plt.axis('off')

    if to_save and len(display_list) == len(settings):
        plt.savefig(os.path.join(MODEL_DIR, f'comparativo.svg'), dpi=300)
        plt.savefig(os.path.join(MODEL_DIR, f'comparativo.png'), dpi=300)

    plt.show()

# Modelo em mémoria

Imports da seção

In [ ]:
from tensorflow.keras.models import model_from_json

In [ ]:
# Based on: https://github.com/mjbhobe/dl-tensorflow-keras/blob/master/kr_helper_funcs.py
def save_model(model, file_name, save_dir):
    """ save the model structure to JSON & weights to HD5 """
    # check if save_dir exists, else create it
    if not os.path.exists(save_dir):
        try:
            mkdir(save_dir)
        except OSError as err:
            print(f'Não foi possível criar o repositório "{save_dir}", para salvar o modelo. Terminando a execução!')
            raise err

    # model structure is saved to $(save_dir)/base_file_name.json
    # weights are saved to $(save_dir)/base_file_name.h5
    model_json = model.to_json()
    json_file_path = os.path.join(save_dir, (file_name + ".json"))
    h5_file_path = os.path.join(save_dir, (file_name + '.h5'))

    with open(json_file_path, "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5\n",
    model.save(
        h5_file_path,
        overwrite=True,
        include_optimizer=True,
        signatures=None,
        options=None,
        save_traces=True,
    )

    print(f'Modelo salvo nos arquivos: "{json_file_path}" e "{h5_file_path}" ')

def load_model(file_name, load_dir):
    """ loads model structure & weights from previously saved state """
    # model structure is loaded $(load_dir)/base_file_name.json
    # weights are loaded from $(load_dir)/base_file_name.h5

    # load model from save_path
    loaded_model = None
    json_file_path = os.path.join(load_dir, (file_name + ".json"))
    h5_file_path = os.path.join(load_dir, (file_name + ".h5"))

    if os.path.exists(json_file_path) and os.path.exists(h5_file_path):
        with open(json_file_path, "r") as json_file:
            loaded_model_json = json_file.read()
            loaded_model = model_from_json(loaded_model_json)
            loaded_model.load_weights(h5_file_path)

        print(f'Modelo construído a partir dos arquivos: "{json_file_path}" e "{h5_file_path}"')

    else:
        print(
            f'Arquivos não encontrados: "{(file_name + ".json")}" e "{(file_name + ".h5")}", na pasta "{load_dir}"'
        )

    return loaded_model

# Métricas manuais

Imports da seção

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
def inter_union_sum(y_true, y_pred):
    # W,H axes of each image
    axes = (1,2)
    
    intersection = K.sum(K.abs(y_pred * y_true), axis=axes)
    mask_sum = K.sum(K.abs(y_true), axis=axes) + K.sum(K.abs(y_pred), axis=axes)
    union = mask_sum  - intersection

    return intersection, union, mask_sum

In [ ]:
def dice_metric(y_true, y_pred):
    # 2*|A & B| / (|A| + |B|)
    intersection, _, mask_sum = inter_union_sum(y_true, y_pred)

    dice = 2 * (intersection + SMOOTH)/(mask_sum + SMOOTH)

    return K.mean(dice)

In [ ]:
def jaccard_metric(y_true, y_pred):
    # |A & B| / (| A U B|)
    intersection, union, _ = inter_union_sum(y_true, y_pred)

    jaccard = (intersection + SMOOTH) / (union + SMOOTH)

    return K.mean(jaccard)

# Arquitetura proposto por Ulas

Imports da seção

In [ ]:
from tensorflow.keras import Input, Model
from tensorflow.keras.activations import swish
from tensorflow.keras.layers import (Activation, AveragePooling2D, BatchNormalization, Conv2D, MaxPooling2D,
                                     UpSampling2D, concatenate)
from tensorflow.keras.models import Sequential

In [ ]:
def _get_ulas_item_dense_layer(inputs, filters, kernel):
    conv = Conv2D(
        filters=filters,
        kernel_size=kernel,
        padding='same',
        use_bias=False
    )(inputs)
    bn = BatchNormalization()(conv)
    act = Activation(swish)(bn)

    return act

In [ ]:
def _get_ulas_dense_layer(input, conv_confs):
    fst = _get_ulas_item_dense_layer(input, *conv_confs[0])
    snd = _get_ulas_item_dense_layer(concatenate([input, fst]), *conv_confs[1])
    trd = _get_ulas_item_dense_layer(concatenate([input, fst, snd]), *conv_confs[2])
    frt = _get_ulas_item_dense_layer(concatenate([input, fst, snd, trd]), *conv_confs[3])

    return frt

In [ ]:
def get_ulas_model(num_classes=1, pool_size=POOL_SIZE):
    input = Input(
        shape=(IMG_SZ, IMG_SZ, INPUT_CHANNELS),
        dtype='float32'
    )

    # 1º parte do encoder
    dense1 = _get_ulas_dense_layer(
        input,
        [
            (160, (3, 7)),
            (112, (3, 7)),
            (144, (9, 7)),
            (80, (3, 11)),
        ]
    )
    trans1 = MaxPooling2D(pool_size=(pool_size, pool_size))(dense1)

    # 2º parte do encoder
    dense2 = _get_ulas_dense_layer(
        trans1, 
        [
            (144, (3, 5)),
            (176, (7, 1)),
            (144, (9, 9)),
            (96, (3, 3)),
        ]
    )
    trans2 = AveragePooling2D(pool_size=(pool_size, pool_size))(dense2)

    # 1º parte da ponte
    dense3 = _get_ulas_dense_layer(
        trans2, 
        [
            (176, (1, 1)),
            (128, (3, 5)),
            (208, (7, 7)),
            (212, (3, 3)),
        ]
    )

    # 2º parte da ponte
    dense4 = _get_ulas_dense_layer(
        dense3, 
        [
            (64, (1, 7)),
            (208, (3, 5)),
            (64, (9, 5)),
            (208, (7, 9)),
        ]
    )

    # 1º parte do decoder
    trans3 = UpSampling2D(size=(pool_size, pool_size))(dense4)
    dense5 = _get_ulas_dense_layer(
        trans3, 
        [
            (208, (5, 7)),
            (64, (3, 5)),
            (96, (11, 11)),
            (112, (7, 7)),
        ]
    )

    # 2º parte do decoder
    trans4 = UpSampling2D(size=(pool_size, pool_size))(dense5)
    dense6 = _get_ulas_dense_layer(
        trans4, 
        [
            (128, (5, 5)),
            (80, (11, 7)),
            (64, (1, 1)),
            (16, (3, 7)),
        ]
    )

    final = Conv2D(
        filters=num_classes,
        kernel_size=(3, 7),
        padding='same',
        activation='softmax',
    )(dense6)

    model = Model(inputs=input, outputs=final)

    return model

In [ ]:
# def get_ulas_model_v2(num_classes=1, pool_size=POOL_SIZE):
#     return get_ulas_model(num_classes, pool_size)

In [ ]:
def get_ulas_model_dice_loss(num_classes=1, pool_size=POOL_SIZE):
    return get_ulas_model(num_classes, pool_size)

# Dependências de Treinamento



Imports da seção

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import MeanIoU
from segmentation_models.losses import DiceLoss, CategoricalFocalLoss
from segmentation_models.metrics import IOUScore, FScore

In [ ]:
optimizer = Adam(
    learning_rate=INIT_LEARNING_RATE
)

In [ ]:
def loss_func(class_weights):
    dice_loss = DiceLoss(class_weights=class_weights, smooth=SMOOTH)
    # focal_loss = CategoricalFocalLoss(gamma=FOCAL_LOSS_GAMA)

    return dice_loss
    # return focal_loss
    # return dice_loss + focal_loss

In [ ]:
def metric_func(class_weights):
    return [
        dice_metric,
        jaccard_metric,
        'accuracy',
        'categorical_accuracy',
        MeanIoU(num_classes=CLASSES_CNT),
        IOUScore(class_weights=class_weights, smooth=SMOOTH, threshold=METRIC_THRESHOLD),
        FScore(smooth=SMOOTH, threshold=METRIC_THRESHOLD)
    ]

In [ ]:
def compile_model(model, class_weights=[]):
    model.compile(
        optimizer=optimizer,
        # 'categorical_crossentropy' | 'sparse_categorical_crossentropy'
        # https://keras.io/api/losses/probabilistic_losses/
        # loss='categorical_crossentropy',
        loss=loss_func(class_weights),
        # https://keras.io/api/metrics/
        metrics=metric_func(class_weights),
        run_eagerly=True
    )

    return model

Callbacks

In [ ]:
epoch_milestone = ModelCheckpoint(
    STEPS_FILENAME,
    monitor=f'val_{EVAL_METRIC}',
    save_best_only=True,
    mode='max',
    verbose=1,
)

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=EARLY_STOP_PATIENCE,
    verbose=1,
)

In [ ]:
csv_logger = CSVLogger(
    CSV_LOG_FILENAME,
    separator=';',
    append=True,
)

In [ ]:
lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    patience=LR_REDUCER_PATIENCE,
    min_lr=MIN_LEARNING_RATE,
    factor=1e-1,
    verbose=1
)

In [ ]:
callbacks = [
    epoch_milestone,
    early_stop,
    csv_logger,
    lr_reducer,
]

# Carregando dados



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_cnt, train_img_filenames = check_cnt(TRAINING_DIR)
train_cnt

In [ ]:
xval_cnt, xval_img_filenames = check_cnt(X_VAL_DIR)
xval_cnt

In [ ]:
test_cnt, test_img_filenames = check_cnt(TEST_DIR)
test_cnt

In [ ]:
there_are_preload_data = all([train_cnt, test_cnt, xval_cnt])
there_are_preload_data

Procedimentos na ausência de dados

In [ ]:
if not there_are_preload_data:
    create_data()

In [ ]:
if not there_are_preload_data:
    # classes_weight = load_classes_weight(raw_train_labels)
    classes_weight = load_classes_weight()
else:
    classes_weight = load_classes_weight()

print(f'classes_weight:\n{classes_weight}')

In [ ]:
if not there_are_preload_data:
    train_cnt, train_img_filenames = check_cnt(TRAINING_DIR)

    xval_cnt, xval_img_filenames = check_cnt(X_VAL_DIR)

    test_cnt, test_img_filenames = check_cnt(TEST_DIR)

In [ ]:
with tf.device('/cpu:0'):
    del train_img_filenames
    train_dataset = get_dataset(TRAINING_DIR, DT_SHUFFLE_SZ, should_data_aug=True)

train_dataset

In [ ]:
with tf.device('/cpu:0'):
    del xval_img_filenames
    xval_dataset = get_dataset(X_VAL_DIR)

xval_dataset

In [ ]:
with tf.device('/cpu:0'):
    del test_img_filenames
    test_dataset = get_dataset(TEST_DIR)

test_dataset

## Visualização teste

In [ ]:
# from random import randint

In [ ]:
# r_ind = randint(0, len(train_gen) - 1)

# img_batch, mask_batch = train_gen.__getitem__(r_ind)

In [ ]:
# r_ind = randint(0, len(img_batch) - 1)

# sample_image, sample_mask = img_batch[r_ind], mask_batch[r_ind]

# print('image', sample_image.dtype, sample_image.shape)
# print('mask', sample_mask.dtype, sample_mask.shape)

# display_sample([sample_image, sample_mask])

# Carregando model

Imports da seção

In [ ]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.utils import plot_model

In [ ]:
get_model = f'get_{MODEL_FILENAME}'

In [ ]:
# Free up RAM in case the model definition cells were run multiple times
clear_session()

In [ ]:
model = load_model(MODEL_FILENAME, MODEL_DIR) or eval(get_model)(CLASSES_CNT)

In [ ]:
model = compile_model(model, classes_weight)

In [ ]:
# save_model(model, MODEL_FILENAME, MODEL_DIR)

In [ ]:
model.summary()

In [ ]:
# get_model_memory_usage(BATCH_SIZE, model)

In [ ]:
# keras_model_memory_usage_in_bytes(model, batch_size=BATCH_SIZE)

In [ ]:
plot_model(
    model,
    to_file=PLOT_FILENAME,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=True,
    dpi=120,
)

# Visualização prévia

In [ ]:
# import matplotlib.pyplot as plt
# from random import randint

In [ ]:
# plt.figure(dpi=180)

In [ ]:
# r_ind = randint(0, len(train_gen) - 1)

# img_batch, mask_batch = train_gen.__getitem__(r_ind)

In [ ]:
# for image, mask in test_dataset.take(1):
#     sample_image, sample_mask = image, mask

# predict_img = tf.convert_to_tensor(model.predict(sample_image))

# print('input', sample_image[0].dtype, sample_image[0].shape)
# print('mask', sample_mask[0].dtype, sample_mask[0].shape)
# print('pred', predict_img[0].dtype, predict_img[0].shape)

# display_sample([sample_image[0], sample_mask[0], predict_img[0]], to_save=True)

# Treinamento

In [ ]:
STEPS_PER_EPOCH = int((train_cnt // BATCH_SIZE) * .4)
DATA = train_dataset

STEPS_PER_EPOCH

In [ ]:
VALIDATION_STEPS = int((xval_cnt // BATCH_SIZE) * .5)
XVAL_DATA = xval_dataset

VALIDATION_STEPS 

In [ ]:
results = model.fit(
    DATA,
    initial_epoch=INITAL_EPOCH,
    epochs=MAX_EPOCH,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=XVAL_DATA,
    validation_steps=VALIDATION_STEPS,
    callbacks=callbacks
)

In [ ]:
save_model(model, MODEL_FILENAME, MODEL_DIR)

In [ ]:
print(results.history)

# Visualização final

In [ ]:
import matplotlib.pyplot as plt
from random import randint

In [ ]:
plt.figure(dpi=180)

In [ ]:
for image, mask in test_dataset.take(1):
    sample_image, sample_mask = image, mask

predict_img = tf.convert_to_tensor(model.predict(sample_image))

display_sample([sample_image[0], sample_mask[0], predict_img[0]], to_save=True)